# A Knowledge based Recommendation System for Project Mangement Ontology learning approach

The procedure of ontology construction can be done in one of three ways: manual construction; cooperative construction (need the human intervention during the constructing process) and (semi-) automatic construction which considered the Ontology Learning (OL) approach. OL from text is the process for acquiring and representing knowledge from text [structured (database), semi-structured (XML file) and unstructured (.txt, pdf, etc)] to be in machine-understandable form [OWL, RDF (Resource Description Framework), or RDFS (Resource Description Framework Schema)], by applying a set of methods and techniques (NLP, data mining, and machine learning).

-Natural Language Processing via NLTK and Spacy matcher linguistic-based preprocessing technique: (1)Tokénization and normalization (2) part-of –speech tagging (POS), (3)posTagger, (4) stopwords, Lemmatization (Stemming), (5) chunking. levenshetein measure, TF-IDF measure, leveithen measure, cosine similarity measure, topic modeling LDA, n-gram, -Recommenadtions techniques based on ML : hiaachical clutering, classification, KNN, etc -Performance measures: Precision, Recall and F measure, -Programmation: Python, java; -Semantic web language/Tool: OWL2, RDF and SWRL

# Imports

In [1]:
import sys, fitz
import pandas as pd
import re
import string
import pandas as pd
import numpy as np

import spacy
from spacy.matcher import Matcher
from spacy.tokens import span
from spacy import displacy


import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, RegexpParser

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 


from nltk.corpus import stopwords

from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.extras import describer
from rdflib.namespace import XSD


nltk.download('conll2000')
from nltk.corpus import conll2000
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from keybert import KeyBERT


#kw_model = KeyBERT(model='all-mpnet-base-v2')


#stop = stopwords.words('english')

#nltk.download('omw-1.4')
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\Farjo\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


# Util Funtions For Text-Preprocessing

In [2]:
def remove_punctuation(text):
    """
        Remove the punctuation
    """
    return re.sub(r'[]"$%&\'()*+/=#@[\\^_`{|}~-]+', " ", text)

In [3]:
def remove_number(text):
    """
        Remove the number
    """
    pattern = r'[0-9]'
    # Match all digits in the string and replace them with an empty string
    new_string = re.sub(pattern, ' ', text)

    return new_string

In [4]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r' ', text)

In [5]:
def remove_lineBreak(text):
    """
        Remove line break
    """
    return re.sub("\n"," ",text)

In [6]:
def lowerCase(text):
    """
        Transform all the text to lower case
    """
    return text.lower()

In [7]:
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

In [8]:
def remove_uu(text):
    """
        Remove the uu  
    """
    return re.sub(r'u u',' ', text)

In [9]:
def find_title(text):
    '''
    Find the title presented in the text with the regex pattern (Exp: 5.1 Plan scope mangement)
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        the title found in the text 
    '''
    return re.findall("[0-9].*[A-Z]\n",text)

In [10]:
def get_section(text):
    '''
    Get the sections in a text (Exp: section 4.1.2)
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        the list of the section founded in the paragraph 
    '''
    return re.findall('Section\s[0-9].{1,7}',text)

In [11]:
def get_figure(text):
    '''
    Get the figures in a text (Exp: section 4.1.2)
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        the list of the figures founded in the paragraph 
    '''
    return re.findall("Figure\s[0-9]-[0-9]{1,2}.\s[A-Z]{1}.*",text)

In [12]:
def get_def(text):
    if re.findall('^Described in Section',text)  :
        return ''
    else : 
        return re.sub('The inputs, tools and techniques, and outputs of this process are depicted in (.*) [0-9]{3}',' ',text)
    #return text

In [13]:
def get_clean_figure(text):
    return re.sub('(Figure\s\d-\d{1,2}.\sFigure\s[0-9].*)', ' ', text) 

In [14]:
def get_process_of_concept(df):
    '''
    Get the topic of each concept  (Exp: concept = PROJECT CHARTER -- topic = PLAN SCOPE MANAGEMENT )
    
    Args:
        df (str): the concept column to which the function is to be applied, string
    
    Returns:
        the topic of the concept passed 
    '''
    index = []
    for i in range(0,len(df['content'])) :
        if df['content'][i] == ' ':
            index.append(i)
    df['topic'] = ''
    for j in range(0,len(index)):
        if j == len(index)-1:
            df['topic'][index[j]:] = df['concept'][index[j]]
            break;
        else :
            df['topic'][index[j]:index[j+1]] = df['concept'][index[j]]
    return df['topic']

In [15]:
def create_columns_from_each_chapter(text,title,content,figures):
    '''
    Creation of the dataframe from the text passed
    
    Args:
        text (str): the text from the pmbok that will be preprocced, string
        title (list): the list that will contain the title 
        content (str): the str that will contain the content
        figures (list): the list that contain the figures
    Returns:
        A dataframe with the following columns : concept, topic, figure, section, type,content 
    '''
    dic = {}
    title = find_title(text)
    for i in range(0,len(title)):
        value = re.sub('\n',' ',title[i])
        title[i] = value.strip()
    
    text = remove_lineBreak(text)
    
    for i in range(0,len(title)):
        if i == (len(title)-1) : 
            content.append(re.findall(title[-1]+'(.*)'+'endchapter',text))
            break;
        else:
            content.append(re.findall(title[i]+'(.*)'+title[i+1],text))
    for i in range(0,len(content)):
        dic[title[i]] = content[i][0]
        
    df = pd.DataFrame(list(dic.items()),columns=['concept','content'])
    
    df['section'] = df['content'].apply(lambda x : get_section(x))
    df['figure'] = None
    for i in range(0,len(figures)):
        df['figure'][i] = figures[i] #df['definition'].apply(lambda x : get_figure(x))
    df['type'] = get_type(df)
    
    
    
    return df #title,content

In [16]:
def get_type(df):
    
    '''
    Get the type of each concept  (Exp: concept = PROJECT CHARTER -- type =  inputs of PLAN SCOPE MANAGEMENT )
    
    Args:
        df (str): the concept column to which the function is to be applied, string
    
    Returns:
        the type of each concept passed 
    '''
    
    start_input = []
    start_tools_and_techniques = []
    start_output = []
    
    title_list = df['concept'].apply(lambda x : lowerCase(x))
    for i in range(0,len(title_list)):
        if 'inputs' in title_list[i] :
            start_input.append(i)
        if 'tools and techniques' in title_list[i] :
            start_tools_and_techniques.append(i)
        if 'outputs' in title_list[i] :
            start_output.append(i)
    for i in range(0,len(start_input)) :
        if i == len(start_input)-1 :
            df.loc[start_input[-1]:start_tools_and_techniques[-1],'type'] = 'inputs'
            df.loc[start_tools_and_techniques[-1]:start_output[-1],'type'] = 'tools_and_techniques'
            df.loc[start_output[-1]:,'type'] = 'outputs'
            break
        df.loc[start_input[i]:start_tools_and_techniques[i],'type'] = 'inputs'
        df.loc[start_tools_and_techniques[i]:start_output[i],'type'] = 'tools_and_techniques'
        df.loc[start_output[i]:start_input[i+1],'type'] = 'outputs'
    return df['type']

In [17]:
def clean_topic(text):
    '''
    remove the unwanted word from the topics
    
    Args:
        text (str): the concept column to which the function is to be applied, string
    
    Returns:
        the cleaned text 
    '''
    if re.findall(': INPUTS',text)  :
        return re.sub(': INPUTS',' ',text)
    elif re.findall(': OUTPUTS',text) :
        return re.sub(': OUTPUTS',' ',text)
    elif re.findall(': TOOLS AND TECHNIQUES',text) :
        return re.sub(': TOOLS AND TECHNIQUES',' ',text)
    else :
        return text

In [18]:
def clean_dataFrame(df,columns):
    
    
    for col in columns:
        df[col] = df[col].apply(lambda x : remove_punctuation(x))
        df[col] = df[col].apply(lambda x : remove_non_ascii(x))
        df[col] = df[col].apply(lambda x : remove_number(x))
        df[col] = df[col].apply(lambda x : remove_extra_whitespaces_func(x))
        df[col] = df[col].apply(lambda x : lowerCase(x))
    
    #df = df.apply(lambda x : remove_point(x))
    return df 

# Import Data From The PDF File

## 1. Retrive The Text From The PMBOK

In [19]:
# Get The text from the pdf file with the library fitz
fdoc = fitz.open("PMBOK6-2017.pdf")
header = "Header"  # text in header
footer = "Page %i of %i"  # text in footer
page = []
for i  in range(0,573):
    page.append(fdoc[i].get_text())  # insert header

## 2. Get the text for each chapter

### 2.1 Initialize Variables

In [20]:
# variable represent each chapter that we will work with 
scope = []
schedule = []
cost = []

# variable represent title that we get from each chapter and the variable with contient the whole text that we will work with 
scope_title = []
scope_title_without_number = []
scope_text = ' '
scope_content = []

schedule_title = []
schedule_title_without_number = []
schedule_text = ' '
schedule_content = []

cost_title = []
cost_title_without_number = []
cost_text = ' '
cost_content = []

figures_scope = []
figures_schedule = []
figures_cost = []

### 2.2 Project scope management

In [21]:
for i in range(164,207):
    p = re.sub('Not For Distribution, Sale or Reproduction.', ' ', page[i])
    p = re.sub('Part 1 - Guide', ' ', p)
    scope.append(p)
    scope_title.append(re.findall("[0-9].*[A-Z]\n",p))
    figures_scope.append(re.findall("Figure\s[0-9]-[0-9]{1,2}.\s[A-Z]{1}.*",p))

In [22]:
for i in range(0,len(scope)):
    scope_text = scope_text + scope[i]
scope_text = scope_text + ' endchapter'

In [23]:
df_scope = create_columns_from_each_chapter(scope_text,scope_title,scope_content,figures_scope)
df_scope['topic'] = ''
df_scope['topic'] = get_process_of_concept(df_scope)
df_scope['content'] = df_scope['content'].apply(lambda x : remove_extra_whitespaces_func(x))
df_scope['def'] = df_scope['content'].apply(lambda x : get_def(x))
df_scope['ref'] = df_scope['section'] + df_scope['figure']


In [24]:
df_scope.head()

,concept,content,section,figure,type,topic,def,ref
0,5.4 Create WBS,—The process of subdividing project deliverabl...,[],[],NaN,,—The process of subdividing project deliverabl...,[]
1,5.1 PLAN SCOPE MANAGEMENT,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2...",[Figure 5-1. Project Scope Management Overview],NaN,,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2..."
2,5.1.1 PLAN SCOPE MANAGEMENT: INPUTS,,[],[],inputs,5.1.1 PLAN SCOPE MANAGEMENT: INPUTS,,[]
3,5.1.1.1 PROJECT CHARTER,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,5.1.1 PLAN SCOPE MANAGEMENT: INPUTS,,[Section 4.1.3.1.]
4,5.1.1.2 PROJECT MANAGEMENT PLAN,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 8.1.3.1.]",[],inputs,5.1.1 PLAN SCOPE MANAGEMENT: INPUTS,,"[Section 4.2.3.1., Section 8.1.3.1.]"


In [25]:
columns_to_clean = ['concept','topic']

df_scope['topic'] = df_scope['topic'].apply(lambda x : clean_topic(x))
df_scope['def'] = df_scope['def'].apply(lambda x : remove_non_ascii(x))
df_scope = clean_dataFrame(df_scope,columns_to_clean)

In [26]:
df_scope.head()

,concept,content,section,figure,type,topic,def,ref
0,. create wbs,—The process of subdividing project deliverabl...,[],[],NaN,,The process of subdividing project deliverabl...,[]
1,. plan scope management,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2...",[Figure 5-1. Project Scope Management Overview],NaN,,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2..."
2,. . plan scope management: inputs,,[],[],inputs,. . plan scope management,,[]
3,. . . project charter,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,. . plan scope management,,[Section 4.1.3.1.]
4,. . . project management plan,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 8.1.3.1.]",[],inputs,. . plan scope management,,"[Section 4.2.3.1., Section 8.1.3.1.]"


In [27]:
df_scope['topic'].unique()

array(['', '. . plan scope management', '. . collect requirements',
       '. . define scope', '. . create wbs', '. . validate scope',
       '. . control scope'], dtype=object)

### 2.3 Project schedule management

In [28]:
for i in range(208,265):
    p = re.sub('Not For Distribution, Sale or Reproduction.', ' ', page[i])
    p = re.sub('Part 1 - Guide', ' ', p) 
    schedule.append(p)
    schedule_title.append(re.findall("[0-9].*[A-Z]\n",p))
    figures_schedule.append(re.findall("Figure\s[0-9]-[0-9]{1,2}.\s[A-Z]{1}.*",p))

In [29]:
for i in range(0,len(schedule)):
    schedule_text = schedule_text + schedule[i]
schedule_text = schedule_text + ' endchapter'

In [30]:
df_schedule = create_columns_from_each_chapter(schedule_text,schedule_title,schedule_content,figures_schedule)
df_schedule['topic'] = ''
df_schedule['topic'] = get_process_of_concept(df_schedule)
df_schedule['content'] = df_schedule['content'].apply(lambda x : remove_extra_whitespaces_func(x))
df_schedule['def'] = df_schedule['content'].apply(lambda x : get_def(x))
df_schedule['ref'] = df_schedule['section'] + df_schedule['figure']


In [31]:
df_schedule.head()

,concept,content,section,figure,type,topic,def,ref
0,6.1 PLAN SCHEDULE MANAGEMENT,Plan Schedule Management is the process of est...,[],[],NaN,,Plan Schedule Management is the process of est...,[]
1,6.1.1 PLAN SCHEDULE MANAGEMENT: INPUTS,,[],[Figure 6-1. Project Schedule Management Overv...,inputs,6.1.1 PLAN SCHEDULE MANAGEMENT: INPUTS,,[Figure 6-1. Project Schedule Management Overv...
2,6.1.1.1 PROJECT CHARTER,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,6.1.1 PLAN SCHEDULE MANAGEMENT: INPUTS,,[Section 4.1.3.1.]
3,6.1.1.2 PROJECT MANAGEMENT PLAN,Described in Section 4.3.2.1. Project manageme...,"[Section 4.3.2.1., Section 5.1.3.1., Section 4...",[Figure 6-2. Scheduling Overview],inputs,6.1.1 PLAN SCHEDULE MANAGEMENT: INPUTS,,"[Section 4.3.2.1., Section 5.1.3.1., Section 4..."
4,6.1.1.3 ENTERPRISE ENVIRONMENTAL FACTORS,The enterprise environmental factors that can ...,[],[],inputs,6.1.1 PLAN SCHEDULE MANAGEMENT: INPUTS,The enterprise environmental factors that can ...,[]


In [32]:
columns_to_clean = ['concept','topic']

df_schedule['topic'] = df_schedule['topic'].apply(lambda x : clean_topic(x))
df_schedule['def'] = df_schedule['def'].apply(lambda x : remove_non_ascii(x))

df_schedule = clean_dataFrame(df_schedule,columns_to_clean)


In [33]:
df_schedule.head()

,concept,content,section,figure,type,topic,def,ref
0,. plan schedule management,Plan Schedule Management is the process of est...,[],[],NaN,,Plan Schedule Management is the process of est...,[]
1,. . plan schedule management: inputs,,[],[Figure 6-1. Project Schedule Management Overv...,inputs,. . plan schedule management,,[Figure 6-1. Project Schedule Management Overv...
2,. . . project charter,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,. . plan schedule management,,[Section 4.1.3.1.]
3,. . . project management plan,Described in Section 4.3.2.1. Project manageme...,"[Section 4.3.2.1., Section 5.1.3.1., Section 4...",[Figure 6-2. Scheduling Overview],inputs,. . plan schedule management,,"[Section 4.3.2.1., Section 5.1.3.1., Section 4..."
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...,[],[],inputs,. . plan schedule management,The enterprise environmental factors that can ...,[]


In [34]:
df_schedule['topic'].unique()

array(['', '. . plan schedule management', '. . define activities',
       '. . sequence activities', '. . estimate activity durations',
       '. . develop schedule', '. . .d', '. . .b', '. . .g', '. . .p',
       '. . control schedule'], dtype=object)

In [35]:
index_p = df_schedule.index[df_schedule['topic']== '. . .p'].tolist()
index_b = df_schedule.index[df_schedule['topic']== '. . .b'].tolist()
index_d = df_schedule.index[df_schedule['topic']== '. . .d'].tolist()
index_g = df_schedule.index[df_schedule['topic']== '. . .g'].tolist()
index = index_p + index_d + index_b + index_g
index

[88, 89, 90, 91, 92, 93, 94, 95, 81, 84, 85, 82, 83, 86, 87]

In [36]:
df_schedule = df_schedule.drop(index = index, axis=1)
df_schedule.head()

,concept,content,section,figure,type,topic,def,ref
0,. plan schedule management,Plan Schedule Management is the process of est...,[],[],NaN,,Plan Schedule Management is the process of est...,[]
1,. . plan schedule management: inputs,,[],[Figure 6-1. Project Schedule Management Overv...,inputs,. . plan schedule management,,[Figure 6-1. Project Schedule Management Overv...
2,. . . project charter,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,. . plan schedule management,,[Section 4.1.3.1.]
3,. . . project management plan,Described in Section 4.3.2.1. Project manageme...,"[Section 4.3.2.1., Section 5.1.3.1., Section 4...",[Figure 6-2. Scheduling Overview],inputs,. . plan schedule management,,"[Section 4.3.2.1., Section 5.1.3.1., Section 4..."
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...,[],[],inputs,. . plan schedule management,The enterprise environmental factors that can ...,[]


In [37]:
df_schedule['topic'].unique()

array(['', '. . plan schedule management', '. . define activities',
       '. . sequence activities', '. . estimate activity durations',
       '. . develop schedule', '. . control schedule'], dtype=object)

### 2.4 Project cost management

In [38]:
for i in range(266,306):
    p = re.sub('Not For Distribution, Sale or Reproduction.', ' ', page[i])
    p = re.sub('Part 1 - Guide', ' ', p)
    cost.append(p)
    cost_title.append(re.findall("[0-9].*[A-Z]\n",p))
    figures_cost.append(re.findall("Figure\s[0-9]-[0-9]{1,2}.\s[A-Z]{1}.*",p))

In [39]:
for i in range(0,len(cost)):
    cost_text = cost_text + cost[i]
cost_text = cost_text + ' endchapter'

In [40]:
df_cost = create_columns_from_each_chapter(cost_text,cost_title,cost_content,figures_cost)
df_cost['topic'] = ''
df_cost['topic'] = get_process_of_concept(df_cost)
df_cost['content'] = df_cost['content'].apply(lambda x : remove_extra_whitespaces_func(x))
df_cost['def'] = df_cost['content'].apply(lambda x : get_def(x))
df_cost['ref'] = df_cost['section'] + df_cost['figure']


In [41]:
df_cost.head()

,concept,content,section,figure,type,topic,def,ref
0,7.1 PLAN COST MANAGEMENT,Plan Cost Management is the process of deﬁning...,[],[],NaN,,Plan Cost Management is the process of deﬁning...,[]
1,7.1.1 PLAN COST MANAGEMENT: INPUTS,,[],[Figure 7-1. Project Cost Management Overview],inputs,7.1.1 PLAN COST MANAGEMENT: INPUTS,,[Figure 7-1. Project Cost Management Overview]
2,7.1.1.1 PROJECT CHARTER,Described in Section 4.2.3.1. The project char...,[Section 4.2.3.1.],[],inputs,7.1.1 PLAN COST MANAGEMENT: INPUTS,,[Section 4.2.3.1.]
3,7.1.1.2 PROJECT MANAGEMENT PLAN,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 6.1.3.1., Section 1...",[],inputs,7.1.1 PLAN COST MANAGEMENT: INPUTS,,"[Section 4.2.3.1., Section 6.1.3.1., Section 1..."
4,7.1.1.3 ENTERPRISE ENVIRONMENTAL FACTORS,The enterprise environmental factors that can ...,[],"[Figure 7-2. Figure 7-3 depicts the data ﬂow ,...",inputs,7.1.1 PLAN COST MANAGEMENT: INPUTS,The enterprise environmental factors that can ...,"[Figure 7-2. Figure 7-3 depicts the data ﬂow ,..."


In [42]:
columns_to_clean = ['concept','topic']

df_cost['topic'] = df_cost['topic'].apply(lambda x : clean_topic(x))
df_cost['def'] = df_cost['def'].apply(lambda x : remove_non_ascii(x))

df_cost = clean_dataFrame(df_cost,columns_to_clean)


In [43]:
df_cost.head()

,concept,content,section,figure,type,topic,def,ref
0,. plan cost management,Plan Cost Management is the process of deﬁning...,[],[],NaN,,Plan Cost Management is the process of de ning...,[]
1,. . plan cost management: inputs,,[],[Figure 7-1. Project Cost Management Overview],inputs,. . plan cost management,,[Figure 7-1. Project Cost Management Overview]
2,. . . project charter,Described in Section 4.2.3.1. The project char...,[Section 4.2.3.1.],[],inputs,. . plan cost management,,[Section 4.2.3.1.]
3,. . . project management plan,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 6.1.3.1., Section 1...",[],inputs,. . plan cost management,,"[Section 4.2.3.1., Section 6.1.3.1., Section 1..."
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...,[],"[Figure 7-2. Figure 7-3 depicts the data ﬂow ,...",inputs,. . plan cost management,The enterprise environmental factors that can ...,"[Figure 7-2. Figure 7-3 depicts the data ﬂow ,..."


In [44]:
df_cost['topic'].unique()

array(['', '. . plan cost management', '. . estimate costs',
       '. . determine budget', '. . control costs'], dtype=object)

### 2.5 Merge All three DataFrame:

In [45]:
final_df = df_scope.append(df_schedule)
final_df = final_df.append(df_cost)
final_df = final_df.drop(index=0,axis=1)
final_df = final_df.reset_index(drop=True)

In [46]:
final_df.head()

,concept,content,section,figure,type,topic,def,ref
0,. plan scope management,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2...",[Figure 5-1. Project Scope Management Overview],NaN,,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2..."
1,. . plan scope management: inputs,,[],[],inputs,. . plan scope management,,[]
2,. . . project charter,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,. . plan scope management,,[Section 4.1.3.1.]
3,. . . project management plan,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 8.1.3.1.]",[],inputs,. . plan scope management,,"[Section 4.2.3.1., Section 8.1.3.1.]"
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...,[],"[Figure 5-2. Figure 5-3 depicts , Figure 5-2. ...",inputs,. . plan scope management,The enterprise environmental factors that can ...,"[Figure 5-2. Figure 5-3 depicts , Figure 5-2. ..."


In [47]:
final_df = final_df.fillna(value='')
final_df

,concept,content,section,figure,type,topic,def,ref
0,. plan scope management,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2...",[Figure 5-1. Project Scope Management Overview],,,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2..."
1,. . plan scope management: inputs,,[],[],inputs,. . plan scope management,,[]
2,. . . project charter,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,. . plan scope management,,[Section 4.1.3.1.]
3,. . . project management plan,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 8.1.3.1.]",[],inputs,. . plan scope management,,"[Section 4.2.3.1., Section 8.1.3.1.]"
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...,[],"[Figure 5-2. Figure 5-3 depicts , Figure 5-2. ...",inputs,. . plan scope management,The enterprise environmental factors that can ...,"[Figure 5-2. Figure 5-3 depicts , Figure 5-2. ..."
...,...,...,...,...,...,...,...,...
245,. . . work performance information,Described in Section 4.5.1.3. Work performance...,"[Section 4.5.1.3., Section 4.5.3.1)]",,outputs,. . control costs,,
246,. . . cost forecasts,Either a calculated EAC value or a bottom-up E...,[],,outputs,. . control costs,Either a calculated EAC value or a bottom-up E...,
247,. . . change requests,Described in Section 4.3.3.4. Analysis of proj...,"[Section 4.3.3.4., Section 4.6). ]",,outputs,. . control costs,,
248,. . . project management plan updates,Any change to the project management plan goes...,"[Section 7.1.3.1., Section 7.3.3.1., Section 4...",,outputs,. . control costs,Any change to the project management plan goes...,


In [48]:
final_df['topic'].unique()

array(['', '. . plan scope management', '. . collect requirements',
       '. . define scope', '. . create wbs', '. . validate scope',
       '. . control scope', '. . plan schedule management',
       '. . define activities', '. . sequence activities',
       '. . estimate activity durations', '. . develop schedule',
       '. . control schedule', '. . plan cost management',
       '. . estimate costs', '. . determine budget', '. . control costs'],
      dtype=object)

In [49]:
final_df['concept'].unique()

array(['. plan scope management', '. . plan scope management: inputs',
       '. . . project charter', '. . . project management plan',
       '. . . enterprise environmental factors',
       '. . . organizational process assets',
       '. . plan scope management: tools and techniques',
       '. . . expert judgment', '. . . data analysis', '. . . meetings',
       '. . plan scope management: outputs',
       '. . . scope management plan',
       '. . . requirements management plan', '. collect requirements',
       '. . collect requirements: inputs', '. . . project documents',
       '. . . business documents', '. . . agreements',
       '. . collect requirements: tools and techniques',
       '. . . data gathering', '. . . decision making',
       '. . . data representation', '. . . interpersonal and team skills',
       '. . . context diagram', '. . . prototypes',
       '. . collect requirements: outputs',
       '. . . requirements documentation',
       '. . . requirements trace

In [50]:
index = final_df.index[final_df['content'] == ''].tolist()
final_df = final_df.drop(index = index, axis=1)

In [51]:
final_df['concept'].unique()

array(['. plan scope management', '. . . project charter',
       '. . . project management plan',
       '. . . enterprise environmental factors',
       '. . . organizational process assets', '. . . expert judgment',
       '. . . data analysis', '. . . meetings',
       '. . . scope management plan',
       '. . . requirements management plan', '. collect requirements',
       '. . . project documents', '. . . business documents',
       '. . . agreements', '. . . data gathering',
       '. . . decision making', '. . . data representation',
       '. . . interpersonal and team skills', '. . . context diagram',
       '. . . prototypes', '. . . requirements documentation',
       '. . . requirements traceability matrix', '. define scope',
       '. . . product analysis', '. . . project scope statement',
       '. . . project documents updates', '. create wbs',
       '. . . decomposition', '. . . scope baseline', '. validate scope',
       '. . . verified deliverables', '. . . work p

In [52]:
final_df.head()

,concept,content,section,figure,type,topic,def,ref
0,. plan scope management,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2...",[Figure 5-1. Project Scope Management Overview],,,Plan Scope Management is the process of creati...,"[Section 4.2.3.1), Section 2.3), an, Section 2..."
2,. . . project charter,Described in Section 4.1.3.1. The project char...,[Section 4.1.3.1.],[],inputs,. . plan scope management,,[Section 4.1.3.1.]
3,. . . project management plan,Described in Section 4.2.3.1. Project manageme...,"[Section 4.2.3.1., Section 8.1.3.1.]",[],inputs,. . plan scope management,,"[Section 4.2.3.1., Section 8.1.3.1.]"
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...,[],"[Figure 5-2. Figure 5-3 depicts , Figure 5-2. ...",inputs,. . plan scope management,The enterprise environmental factors that can ...,"[Figure 5-2. Figure 5-3 depicts , Figure 5-2. ..."
5,. . . organizational process assets,The organizational process assets that can inﬂ...,[],[],inputs,. . plan scope management,The organizational process assets that can in ...,[]


In [53]:
final_df['content'][0]

'Plan Scope Management is the process of creating a scope management plan that documents how the project and product scope will be deﬁned, validated, and controlled. The key beneﬁt of this process is that it provides guidance and direction on how scope will be managed throughout the project. This process is performed once or at predeﬁned points in the project. The inputs, tools and techniques, and outputs of this process are depicted in Figure 5-2. Figure 5-3 depicts the data ﬂow diagram of the process. Figure 5-2. Plan Scope Management: Inputs, Tools & Techniques, and Outputs Figure 5-3. Plan Scope Management: Data Flow Diagram Tools & Techniques Inputs Outputs Plan Scope Management .1 Expert judgment .2 Data analysis • Alternatives analysis .3 Meetings .1 Project charter .2 Project management plan • Quality management plan • Project life cycle description • Development approach .3 Enterprise environmental factors .4 Organizational process assets .1 Scope management plan .2 Requiremen

# Text Preprocessing

## 1. Prepare the data ( Removing stop words, lowercasing ,etc....)

In [54]:
data_to_process = final_df[['concept','def']]
data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('include but are not limited to','include',x))
data_to_process.head()

C:\Users\Farjo\AppData\Local\Temp/ipykernel_10172/3932753283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('include but are not limited to','include',x))


,concept,def
0,. plan scope management,Plan Scope Management is the process of creati...
2,. . . project charter,
3,. . . project management plan,
4,. . . enterprise environmental factors,The enterprise environmental factors that can ...
5,. . . organizational process assets,The organizational process assets that can in ...


In [55]:
# remove bias u u 
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_uu(x))
# lower cassing 
data_to_process['def'] = data_to_process['def'].apply(lambda x : lowerCase(x))
# remove number
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_number(x))
# remove line break 
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_lineBreak(x))
# remove extra white space 
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_extra_whitespaces_func(x))
# remove non ascii text
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_non_ascii(x))
# remove non ascii text
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_punctuation(x))
# remove extra white space 
data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_extra_whitespaces_func(x))

data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('(section [\.{,6}\s\.]{,6})',' ',x))
data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('that can in uence','influence',x))
data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('scope','Scope',x))
data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('de ned','defined',x))
data_to_process['def'] = data_to_process['def'].apply(lambda x : re.sub('bene t','benefit',x))

C:\Users\Farjo\AppData\Local\Temp/ipykernel_10172/4051881670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_process['def'] = data_to_process['def'].apply(lambda x : remove_uu(x))
C:\Users\Farjo\AppData\Local\Temp/ipykernel_10172/4051881670.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_process['def'] = data_to_process['def'].apply(lambda x : lowerCase(x))
C:\Users\Farjo\AppData\Local\Temp/ipykernel_10172/4051881670.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

## 2. Extract each sentence from def

In [56]:
# intialize nlp
nlp = spacy.load("en_core_web_sm")

In [57]:
def extract_sentence(text):
    sentence = []
    doc = nlp(text)
    for sent in doc.sents :
        sentence.append(sent.text)
    return sentence

In [58]:
def lemm(text):
    stop = stopwords.words('english')
    text = nlp(text)
    l = []
    for word in text:
        lemma = word.lemma_
        l.append(lemma)
    return l

In [59]:
data_to_process.head()

,concept,def
0,. plan scope management,plan Scope management is the process of creati...
2,. . . project charter,
3,. . . project management plan,
4,. . . enterprise environmental factors,the enterprise environmental factors influence...
5,. . . organizational process assets,the organizational process assets influence th...


In [60]:
data_to_process['lemma_def'] = data_to_process['def'].apply(lambda x : ' '.join(lemm(x)))
data_to_process['sentence'] = data_to_process['lemma_def'].apply(lambda x : extract_sentence(x))

C:\Users\Farjo\AppData\Local\Temp/ipykernel_10172/1110491310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_process['lemma_def'] = data_to_process['def'].apply(lambda x : ' '.join(lemm(x)))
C:\Users\Farjo\AppData\Local\Temp/ipykernel_10172/1110491310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_process['sentence'] = data_to_process['lemma_def'].apply(lambda x : extract_sentence(x))


In [61]:
data_to_process.head()

,concept,def,lemma_def,sentence
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be the process of creat...
2,. . . project charter,,,[]
3,. . . project management plan,,,[]
4,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[the enterprise environmental factor influence...
5,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[the organizational process asset influence th...


In [62]:
data_to_process['def'][0]

'plan Scope management is the process of creating a Scope management plan that documents how the project and product Scope will be defined, validated, and controlled. the key benefit of this process is that it provides guidance and direction on how Scope will be managed throughout the project. this process is performed once or at predefined points in the project. the Scope management plan is a component of the project or program management plan that describes how the Scope will be defined, developed, monitored, controlled, and validated. the development of the Scope management plan and the detailing of the project Scope begin with the analysis of information contained in the project charter  , the latest approved subsidiary plans of the project management plan  , historical information contained in the organizational process assets  and any other relevant enterprise environmental factors  '

In [63]:
data_to_process['sentence'][0]

['plan Scope management be the process of create a Scope management plan that document how the project and product Scope will be define , validate , and control .',
 'the key benefit of this process be that it provide guidance and direction on how Scope will be manage throughout the project .',
 'this process be perform once or at predefine point in the project .',
 'the Scope management plan be a component of the project or program management plan that describe how the Scope will be define , develop , monitor , control , and validate .',
 'the development of the Scope management plan and the detailing of the project Scope begin with the analysis of information contain in the project charter   , the late approve subsidiary plan of the project management plan   , historical information contain in the organizational process asset   and any other relevant enterprise environmental factor  ']

## 3. Extract verbs and noun with Chuncking

In [64]:
'''
the following funtion represent the chunking model that will be used for the 
relation extraction between the concept
'''

def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff
#Define the chunker class
class NGramTagChunker(ChunkParserI):
    def __init__(self,train_sentences,tagger_classes=[UnigramTagger,BigramTagger]):
        train_sent_tags=conll_tag_chunks(train_sentences)
        self.chunk_tagger=combined_tagger(train_sent_tags,tagger_classes)
    def parse(self,tagged_sentence):
        if not tagged_sentence:
            return None
        pos_tags=[tag for word, tag in tagged_sentence]
        chunk_pos_tags=self.chunk_tagger.tag(pos_tags)
        chunk_tags=[chunk_tag for (pos_tag,chunk_tag) in chunk_pos_tags]
        wpc_tags=[(word,pos_tag,chunk_tag) for ((word,pos_tag),chunk_tag) in zip(tagged_sentence,chunk_tags)]
        return conlltags2tree(wpc_tags)

In [65]:
#Here conll2000 corpus for training shallow parser model
data= conll2000.chunked_sents()
train_data=data[:10900]
test_data=data[10900:]

from nltk.chunk.util import tree2conlltags,conlltags2tree
wtc=tree2conlltags(train_data[1])
wtc
tree=conlltags2tree(wtc)

#train chunker model
ntc=NGramTagChunker(train_data)
#evaluate chunker model performance
print(ntc.evaluate(test_data))

ChunkParse score:
    IOB Accuracy:  90.0%%
    Precision:     82.1%%
    Recall:        86.3%%
    F-Measure:     84.1%%


In [66]:
t = tuple()
def get_the_rigth_chunking(chunk_tree):
    try :
        begin = len(chunk_tree)-1
        for i in range(begin,0,-1):
            if type(chunk_tree[i]) == type(t):
                chunk_tree.pop(i)
    except IndexError:
        return chunk_tree
            
    return chunk_tree

In [67]:
data_to_process = data_to_process.reset_index(drop=True)
data_to_process.head()

,concept,def,lemma_def,sentence
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be the process of creat...
1,. . . project charter,,,[]
2,. . . project management plan,,,[]
3,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[the enterprise environmental factor influence...
4,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[the organizational process asset influence th...


In [68]:
data_to_process['sentence'][0][0]

'plan Scope management be the process of create a Scope management plan that document how the project and product Scope will be define , validate , and control .'

In [69]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')

def extract_key_words(text):
    keywords = kw_model.extract_keywords(text, 

                                        keyphrase_ngram_range=(2, 3), 

                                         stop_words='english', 

                                         highlight=False,

                                         top_n=5)
    keywords_list= list(dict(keywords).keys())
    return keywords_list

#print(keywords_list)

In [70]:
data_to_process['kw_bert'] = [[] for _ in range(len(data_to_process))]

In [71]:
for i in range(0,len(data_to_process['sentence'])):
    for j in range(0,len(data_to_process['sentence'][i])):
        data_to_process['kw_bert'][i].append(extract_key_words(data_to_process['sentence'][i][j]))


In [72]:
data_to_process['kw_bert'][0]

[['plan scope management',
  'scope management plan',
  'plan scope',
  'scope management process',
  'scope management'],
 ['scope manage project',
  'process provide guidance',
  'direction scope manage',
  'scope manage',
  'benefit process provide'],
 ['process perform predefine',
  'process perform',
  'predefine point project',
  'perform predefine point',
  'perform predefine'],
 ['scope management plan',
  'plan scope define',
  'management plan scope',
  'plan scope',
  'scope management'],
 ['scope management plan',
  'project scope begin',
  'project scope',
  'detailing project scope',
  'scope management']]

In [73]:
data_to_process['sentence'][0], 

(['plan Scope management be the process of create a Scope management plan that document how the project and product Scope will be define , validate , and control .',
  'the key benefit of this process be that it provide guidance and direction on how Scope will be manage throughout the project .',
  'this process be perform once or at predefine point in the project .',
  'the Scope management plan be a component of the project or program management plan that describe how the Scope will be define , develop , monitor , control , and validate .',
  'the development of the Scope management plan and the detailing of the project Scope begin with the analysis of information contain in the project charter   , the late approve subsidiary plan of the project management plan   , historical information contain in the organizational process asset   and any other relevant enterprise environmental factor  '],)

In [74]:
stop = stopwords.words('english')
stop.remove('be')
stop.remove('have')
stop.remove('not')
stop.remove('will')
stop.append('to')
stop.append('not')
stop.append('that')
stop.append('may')

In [75]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'been', 'being', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't

In [76]:
sent = ' '.join(word for word in data_to_process['sentence'][0][0].split() if word not in stop)
sent

'plan Scope management be process create Scope management plan document project product Scope will be define , validate , control .'

In [77]:
data_to_process['sentence'][0][0]

'plan Scope management be the process of create a Scope management plan that document how the project and product Scope will be define , validate , and control .'

In [78]:
from spacy import displacy

doc = nlp(sent)
# in Jupyter Notebook
displacy.render(doc, style="dep", options={'compact': True, 'distance': 120})

In [79]:
doc = nlp(data_to_process['sentence'][0][0])
# in Jupyter Notebook
displacy.render(doc, style="dep", options={'compact': True, 'distance': 120})

In [80]:
#data_to_process['lemma_sentence'] = data_to_process['lemma_sentence'].apply(lambda x : remove_punctuation(x) for i in len(x) )
for i in range(0,len(data_to_process['sentence'])):
    for j in range(0,len(data_to_process['sentence'][i])):
        
        data_to_process['sentence'][i][j] = remove_punctuation(data_to_process['sentence'][i][j])
        data_to_process['sentence'][i][j] = remove_number(data_to_process['sentence'][i][j])
        data_to_process['sentence'][i][j] = remove_extra_whitespaces_func(data_to_process['sentence'][i][j])
        data_to_process['sentence'][i][j] = ' '.join(word for word in data_to_process['sentence'][i][j].split() if word not in stop)

In [81]:
data_to_process.head()

,concept,def,lemma_def,sentence,kw_bert
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be process create Scope...,"[[plan scope management, scope management plan..."
1,. . . project charter,,,[],[]
2,. . . project management plan,,,[],[]
3,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[enterprise environmental factor influence pla...,"[[plan scope management, plan scope, scope man..."
4,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[organizational process asset influence plan S...,"[[plan scope management, scope management proc..."


In [82]:
t = tuple()  
def get_relation_from_definition(text,NP,VB):
    nltk_pos_tagged = nltk.pos_tag(text.split())
    chunk_tree = ntc.parse(nltk_pos_tagged)
    chunk_tree = get_the_rigth_chunking(chunk_tree)
    try :
        for k in range(len(chunk_tree)):
            chunk = chunk_tree[k]
            #print(chunk)
            if chunk.label() == 'NP':
                noun = ' '.join(word for word, pos in chunk.leaves())
                NP.append(noun)
            elif chunk.label() == 'VP':
                verb = ' '.join(word for word, pos in chunk.leaves())
                VB.append(verb)
    except AttributeError:
        pass
    return NP,VB

In [83]:
t = tuple()
NP1=[]
VB=[]
NP2=[]   
def Extract_relation_Chunking(text):        
        
    nltk_pos_tagged = nltk.pos_tag(text.split())
    chunk_tree = ntc.parse(nltk_pos_tagged)
    chunk_tree = get_the_rigth_chunking(chunk_tree)
    try:
        for k in range(len(chunk_tree)):
            if k == (len(chunk_tree) - 3):

                np1 = chunk_tree[-3]
                vb = chunk_tree[-2]
                np2 = chunk_tree[-1]
                if (np1 == 'NP') and (vb == 'VP') and (np2) == 'NP':
                    t1 = ' '.join(word for word, pos in np1.leaves())
                    NP1.append(t1)
                    t2 = ' '.join(word for word, pos in vb.leaves())
                    VB.append(t2)
                    t3 = ' '.join(word for word, pos in np2.leaves())
                    NP2.append(t3)
                    break;
                else:
                    break;
            elif (chunk_tree[k].label() == 'NP') and (chunk_tree[k+1].label() == 'VP') and (chunk_tree[k+2].label() == 'NP') :
                
                np1 = chunk_tree[k]
                vb = chunk_tree[k+1]
                np2 = chunk_tree[k+2]

                t1 = ' '.join(word for word, pos in np1.leaves())
                NP1.append(t1)
                t2 = ' '.join(word for word, pos in vb.leaves())
                VB.append(t2)
                t3 = ' '.join(word for word, pos in np2.leaves())
                NP2.append(t3)
    except IndexError:
        pass
    except AttributeError:
        pass

    return NP1,VB,NP2

In [84]:
for i in range(0,len(data_to_process['sentence'])):
    #NP=[]
    #VB=[]
    for j in range(0,len(data_to_process['sentence'][i])):
        if data_to_process['sentence'][i][j] == '':
            pass
        else:
            NP1,VP,NP2 = Extract_relation_Chunking(data_to_process['sentence'][i][j])
    #data_to_process['noun'][i] = NP 
    #data_to_process['verb'][i] = VP

In [85]:
import pandas as pd
relation_df = pd.DataFrame(list(zip(NP1,VP,NP2)),columns=['noun1','VB','noun2'])
relation_df[15:30]

,noun1,VB,noun2
15,project team,attend,project meeting
16,attendee,include,project manager
17,Scope management plan,be,component project management plan
18,component project management plan,describe,Scope
19,Scope,will be,define
20,define,develop,monitor
21,component Scope management plan,include,process prepare project Scope statement
22,enable creation wbs,detailed,project Scope statement
23,project Scope statement,establish,Scope baseline
24,Scope baseline,will be,maintain


In [86]:
len(relation_df)

908

In [87]:
data_to_process['noun'] = [[] for _ in range(len(data_to_process))]
data_to_process['verb'] = [[] for _ in range(len(data_to_process))]

In [88]:
data_to_process.head()

,concept,def,lemma_def,sentence,kw_bert,noun,verb
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be process create Scope...,"[[plan scope management, scope management plan...",[],[]
1,. . . project charter,,,[],[],[],[]
2,. . . project management plan,,,[],[],[],[]
3,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[enterprise environmental factor influence pla...,"[[plan scope management, plan scope, scope man...",[],[]
4,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[organizational process asset influence plan S...,"[[plan scope management, scope management proc...",[],[]


In [89]:
for i in range(0,len(data_to_process['sentence'])):
    NP=[]
    VB=[]
    for j in range(0,len(data_to_process['sentence'][i])):
        if data_to_process['sentence'][i][j] == '':
            pass
        else:
            NP,VP = get_relation_from_definition(data_to_process['sentence'][i][j],NP,VB)
    data_to_process['noun'][i] = NP 
    data_to_process['verb'][i] = VP

In [90]:
data_to_process.head()

,concept,def,lemma_def,sentence,kw_bert,noun,verb
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be process create Scope...,"[[plan scope management, scope management plan...","[plan Scope management, process, Scope managem...","[be, create, will be, be, will be, be, be, des..."
1,. . . project charter,,,[],[],[],"[be, create, will be, be, will be, be, be, des..."
2,. . . project management plan,,,[],[],[],"[be, create, will be, be, will be, be, be, des..."
3,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[enterprise environmental factor influence pla...,"[[plan scope management, plan scope, scope man...",[enterprise environmental factor influence pla...,[include]
4,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[organizational process asset influence plan S...,"[[plan scope management, scope management proc...",[organizational process asset influence plan S...,[include]


In [91]:
verb_list = []
for i in range(0, len(data_to_process['verb'])-1):
    verb_list +=  data_to_process['verb'][i]

In [92]:
verb_list

['be',
 'create',
 'will be',
 'be',
 'will be',
 'be',
 'be',
 'describe',
 'will be',
 'develop',
 'detailing',
 'begin',
 'project',
 'contain',
 'be',
 'create',
 'will be',
 'be',
 'will be',
 'be',
 'be',
 'describe',
 'will be',
 'develop',
 'detailing',
 'begin',
 'project',
 'contain',
 'be',
 'create',
 'will be',
 'be',
 'will be',
 'be',
 'be',
 'describe',
 'will be',
 'develop',
 'detailing',
 'begin',
 'project',
 'contain',
 'include',
 'include',
 'include',
 'be',
 'include be',
 'be',
 'attend',
 'develop',
 'include',
 'be',
 'describe',
 'will be',
 'develop',
 'include',
 'detailed',
 'establish',
 'will be',
 'will be obtain',
 'be',
 'be',
 'describe',
 'will be',
 'include',
 'will be',
 'will be',
 'will be analyze',
 'will be',
 'require',
 'will be',
 'be',
 'documenting',
 'need',
 'be',
 'be',
 'guide speci',
 'provide',
 'be directly',
 'take',
 'documenting',
 'include',
 'be require be',
 'include',
 'need be',
 'be include',
 'be',
 'become',
 'be',
 '

In [93]:
verbs = []
for word in verb_list:
    split = word.split()
    for w in split :
        if w.lower() not in stop:
            verbs.append(w)

In [94]:
from collections import Counter
counter = Counter(verb_list)
sorted_counter = counter.most_common()
sorted_counter

[('be', 728),
 ('include', 155),
 ('will be', 45),
 ('learn', 43),
 ('approve', 32),
 ('develop', 29),
 ('have', 29),
 ('provide', 28),
 ('remain', 27),
 ('require', 24),
 ('be perform', 24),
 ('earn', 23),
 ('need', 20),
 ('start', 18),
 ('scheduling', 16),
 ('give', 15),
 ('establish', 13),
 ('be often', 12),
 ('refer', 11),
 ('improve', 11),
 ('cost', 11),
 ('identify', 10),
 ('expect', 10),
 ('be apply', 10),
 ('indicate', 10),
 ('begin', 9),
 ('contain', 9),
 ('be include', 9),
 ('ensure', 9),
 ('make', 9),
 ('use', 9),
 ('describe', 8),
 ('involve', 8),
 ('reduce', 8),
 ('be develop', 8),
 ('be require', 8),
 ('create', 7),
 ('include be', 7),
 ('take', 7),
 ('need be', 7),
 ('obtain', 7),
 ('speci', 7),
 ('c', 7),
 ('be exactly', 7),
 ('documenting', 6),
 ('be require be', 6),
 ('be need', 6),
 ('show', 6),
 ('monitoring', 6),
 ('go', 6),
 ('planning', 6),
 ('achieve', 6),
 ('versus', 6),
 ('must be', 6),
 ('project', 5),
 ('have multiple', 5),
 ('be generally', 5),
 ('be someti

In [95]:
noun_list = []
for i in range(0, len(data_to_process['noun'])-1):
    noun_list +=  data_to_process['noun'][i]

In [96]:
noun = []
for word in noun_list:
    if len(word.split()) >= 3:
        noun.append(word)       

In [97]:
noun

['plan Scope management',
 'Scope management plan document project product Scope',
 'key benefit process',
 'provide guidance direction Scope',
 'perform predefine point project',
 'Scope management plan',
 'component project program management plan',
 'development Scope management plan',
 'analysis information contain project charter',
 'late approve subsidiary plan',
 'organizational process asset relevant enterprise environmental factor',
 'enterprise environmental factor influence plan Scope management process',
 'organizational process asset influence plan Scope management process',
 'historical information lesson learn repository',
 'data analysis technique',
 'limit alternative analysis',
 'various way collect requirement',
 'elaborate project product Scope',
 'Scope management plan',
 'select project team member',
 'anyone responsibility Scope management process',
 'Scope management plan',
 'component project management plan',
 'component Scope management plan',
 'process prepa

In [98]:
from collections import Counter

counter = Counter(noun)
sorted_counter_noun = counter.most_common()
sorted_counter_noun

[('key benefit process', 13),
 ('consider input process', 12),
 ('requirement traceability matrix', 9),
 ('update result carry process', 8),
 ('later phase project', 7),
 ('realistic basis performance measurement', 7),
 ('schedule management plan', 7),
 ('Scope management plan', 6),
 ('component project management plan', 6),
 ('datum analysis technique', 6),
 ('project Scope statement', 5),
 ('example project document', 5),
 ('lesson learn register', 5),
 ('use process include', 5),
 ('change cost baseline', 5),
 ('project schedule network diagram', 5),
 ('project management information system', 5),
 ('cost management plan', 5),
 ('perform predefine point project', 4),
 ('data analysis technique', 4),
 ('limit alternative analysis', 4),
 ('change project management plan', 4),
 ('organization change control process', 4),
 ('component require change request project management plan', 4),
 ('performance measurement baseline', 4),
 ('rise individual project risk impact project schedule', 4)

In [99]:
unique_noun = []
for item in sorted_counter_noun:
    unique_noun.append(item[0])

In [100]:
unique_noun

['key benefit process',
 'consider input process',
 'requirement traceability matrix',
 'update result carry process',
 'later phase project',
 'realistic basis performance measurement',
 'schedule management plan',
 'Scope management plan',
 'component project management plan',
 'datum analysis technique',
 'project Scope statement',
 'example project document',
 'lesson learn register',
 'use process include',
 'change cost baseline',
 'project schedule network diagram',
 'project management information system',
 'cost management plan',
 'perform predefine point project',
 'data analysis technique',
 'limit alternative analysis',
 'change project management plan',
 'organization change control process',
 'component require change request project management plan',
 'performance measurement baseline',
 'rise individual project risk impact project schedule',
 'project management information system pmis describe',
 'component work package',
 'value plan value',
 'value actual cost',
 'hi

In [101]:
len(unique_noun)

1465

## 4. Extract verbs and noun with spaCy

In [102]:
data_to_process.head()

,concept,def,lemma_def,sentence,kw_bert,noun,verb
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be process create Scope...,"[[plan scope management, scope management plan...","[plan Scope management, process, Scope managem...","[be, create, will be, be, will be, be, be, des..."
1,. . . project charter,,,[],[],[],"[be, create, will be, be, will be, be, be, des..."
2,. . . project management plan,,,[],[],[],"[be, create, will be, be, will be, be, be, des..."
3,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[enterprise environmental factor influence pla...,"[[plan scope management, plan scope, scope man...",[enterprise environmental factor influence pla...,[include]
4,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[organizational process asset influence plan S...,"[[plan scope management, scope management proc...",[organizational process asset influence plan S...,[include]


In [103]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
for i in range(len(data_to_process['sentence'][0])):
    text = data_to_process['sentence'][0][i]
    text = ' '.join([word for word in text.split() if word not in stop])
    doc = nlp(text)

    # Analyze syntax
    print(f"text {i+1}:",text)
    print('---------------------------------------')
    print(f"Noun phrases {i+1}:", [chunk.text for chunk in doc.noun_chunks])
    print('---------------------------------------')
    print(f"Verbs {i+1}: ", [token.lemma_ for token in doc if token.pos_ == "VERB"])



text 1: plan Scope management be process create Scope management plan document project product Scope will be define , validate , control .
---------------------------------------
Noun phrases 1: ['plan Scope management', 'process', 'Scope management plan document project product Scope']
---------------------------------------
Verbs 1:  ['create']
text 2: key benefit process be provide guidance direction Scope will be manage throughout project .
---------------------------------------
Noun phrases 2: ['key benefit process', 'guidance direction Scope', 'project']
---------------------------------------
Verbs 2:  ['provide', 'manage']
text 3: process be perform predefine point project .
---------------------------------------
Noun phrases 3: ['process', 'predefine point project']
---------------------------------------
Verbs 3:  ['perform']
text 4: Scope management plan be component project program management plan describe Scope will be define , develop , monitor , control , validate .
--

In [104]:
data_to_process['noun_spacy'] = [[] for _ in range(len(data_to_process))]
data_to_process['verb_spacy'] = [[] for _ in range(len(data_to_process))]

for i in range(0,len(data_to_process['sentence'])):
    noun_spacy = []
    verb_spacy = []
    for j in range(len(data_to_process['sentence'][i])):
        text = data_to_process['sentence'][i][j]
        text = ' '.join([word for word in text.split() if word not in stop])
        doc = nlp(text)

        NP = [chunk.text for chunk in doc.noun_chunks]
        noun_spacy += NP
        VB = [token.lemma_ for token in doc if token.pos_ == "VERB"]
        verb_spacy += VB
    data_to_process['noun_spacy'][i] = noun_spacy 
    data_to_process['verb_spacy'][i] = verb_spacy


In [105]:
data_to_process.head()

,concept,def,lemma_def,sentence,kw_bert,noun,verb,noun_spacy,verb_spacy
0,. plan scope management,plan Scope management is the process of creati...,plan Scope management be the process of create...,[plan Scope management be process create Scope...,"[[plan scope management, scope management plan...","[plan Scope management, process, Scope managem...","[be, create, will be, be, will be, be, be, des...","[plan Scope management, process, Scope managem...","[create, provide, manage, perform, describe, d..."
1,. . . project charter,,,[],[],[],"[be, create, will be, be, will be, be, be, des...",[],[]
2,. . . project management plan,,,[],[],[],"[be, create, will be, be, will be, be, be, des...",[],[]
3,. . . enterprise environmental factors,the enterprise environmental factors influence...,the enterprise environmental factor influence ...,[enterprise environmental factor influence pla...,"[[plan scope management, plan scope, scope man...",[enterprise environmental factor influence pla...,[include],[enterprise environmental factor influence pla...,[include]
4,. . . organizational process assets,the organizational process assets influence th...,the organizational process asset influence the...,[organizational process asset influence plan S...,"[[plan scope management, scope management proc...",[organizational process asset influence plan S...,[include],[organizational process asset influence plan S...,"[include, learn]"


In [106]:
print(data_to_process['sentence'][0])

['plan Scope management be process create Scope management plan document project product Scope will be define , validate , control .', 'key benefit process be provide guidance direction Scope will be manage throughout project .', 'process be perform predefine point project .', 'Scope management plan be component project program management plan describe Scope will be define , develop , monitor , control , validate .', 'development Scope management plan detailing project Scope begin analysis information contain project charter , late approve subsidiary plan project management plan , historical information contain organizational process asset relevant enterprise environmental factor']


In [107]:
print(data_to_process['noun'][0])

['plan Scope management', 'process', 'Scope management plan document project product Scope', 'define', 'validate', 'control', 'key benefit process', 'provide guidance direction Scope', 'manage', 'project', 'process', 'perform predefine point project', 'Scope management plan', 'component project program management plan', 'Scope', 'define', 'monitor', 'control', 'validate', 'development Scope management plan', 'project Scope', 'analysis information contain project charter', 'late approve subsidiary plan', 'management plan', 'historical information', 'organizational process asset relevant enterprise environmental factor']


In [108]:
print(data_to_process['noun_spacy'][0])

['plan Scope management', 'process', 'Scope management plan document project product Scope', 'key benefit process', 'guidance direction Scope', 'project', 'process', 'predefine point project', 'Scope management plan', 'component project program management plan', 'Scope', 'development Scope management plan', 'project Scope', 'analysis information', 'project charter', 'late approve subsidiary plan project management plan', 'historical information', 'organizational process', 'relevant enterprise environmental factor']


In [109]:
print(data_to_process['verb'][0])

['be', 'create', 'will be', 'be', 'will be', 'be', 'be', 'describe', 'will be', 'develop', 'detailing', 'begin', 'project', 'contain']


In [110]:
print(data_to_process['verb_spacy'][0])

['create', 'provide', 'manage', 'perform', 'describe', 'develop', 'monitor', 'detail', 'begin', 'contain', 'approve', 'contain', 'asset']


In [111]:
len(data_to_process['noun_spacy'][0]),len(data_to_process['noun'][0])

(19, 26)

In [112]:
data_to_process['def'][0]

'plan Scope management is the process of creating a Scope management plan that documents how the project and product Scope will be defined, validated, and controlled. the key benefit of this process is that it provides guidance and direction on how Scope will be managed throughout the project. this process is performed once or at predefined points in the project. the Scope management plan is a component of the project or program management plan that describes how the Scope will be defined, developed, monitored, controlled, and validated. the development of the Scope management plan and the detailing of the project Scope begin with the analysis of information contained in the project charter  , the latest approved subsidiary plans of the project management plan  , historical information contained in the organizational process assets  and any other relevant enterprise environmental factors  '

In [113]:
verb_list_spacy = []
for i in range(0, len(data_to_process['verb_spacy'])-1):
    verb_list_spacy +=  data_to_process['verb_spacy'][i]

In [114]:
from collections import Counter
counter = Counter(verb_list_spacy)
sorted_counter_verb = counter.most_common()
sorted_counter_verb

[('include', 169),
 ('describe', 130),
 ('use', 80),
 ('provide', 45),
 ('be', 45),
 ('perform', 37),
 ('learn', 36),
 ('require', 36),
 ('start', 35),
 ('determine', 34),
 ('have', 34),
 ('develop', 33),
 ('need', 23),
 ('consider', 23),
 ('remain', 23),
 ('contain', 22),
 ('register', 19),
 ('indicate', 18),
 ('represent', 17),
 ('approve', 16),
 ('refer', 16),
 ('make', 16),
 ('create', 15),
 ('monitor', 15),
 ('establish', 15),
 ('expect', 15),
 ('show', 15),
 ('estimate', 15),
 ('manage', 14),
 ('relate', 14),
 ('identify', 12),
 ('know', 12),
 ('apply', 12),
 ('improve', 12),
 ('give', 12),
 ('begin', 11),
 ('take', 11),
 ('allow', 11),
 ('meet', 11),
 ('accept', 11),
 ('update', 11),
 ('exist', 11),
 ('cost', 11),
 ('define', 10),
 ('carry', 10),
 ('decompose', 10),
 ('compare', 9),
 ('reduce', 9),
 ('change', 9),
 ('become', 8),
 ('bring', 8),
 ('evaluate', 8),
 ('see', 8),
 ('add', 8),
 ('deliver', 8),
 ('authorize', 8),
 ('adjust', 8),
 ('limit', 7),
 ('involve', 7),
 ('ensur

In [115]:
most_important_verb_spacy = []
for item in sorted_counter_verb:
    if 5 < item[1]:
        most_important_verb_spacy.append(item[0])

In [116]:
most_important_verb_spacy

['include',
 'describe',
 'use',
 'provide',
 'be',
 'perform',
 'learn',
 'require',
 'start',
 'determine',
 'have',
 'develop',
 'need',
 'consider',
 'remain',
 'contain',
 'register',
 'indicate',
 'represent',
 'approve',
 'refer',
 'make',
 'create',
 'monitor',
 'establish',
 'expect',
 'show',
 'estimate',
 'manage',
 'relate',
 'identify',
 'know',
 'apply',
 'improve',
 'give',
 'begin',
 'take',
 'allow',
 'meet',
 'accept',
 'update',
 'exist',
 'cost',
 'define',
 'carry',
 'decompose',
 'compare',
 'reduce',
 'change',
 'become',
 'bring',
 'evaluate',
 'see',
 'add',
 'deliver',
 'authorize',
 'adjust',
 'limit',
 'involve',
 'ensure',
 'decide',
 'build',
 'organize',
 'integrate',
 'calculate',
 'maintain',
 'analyze',
 'agree',
 'affect',
 'produce',
 'work',
 'increase',
 'specify',
 'publish',
 'depend',
 'finish',
 'occur',
 'bac']

In [117]:
noun_list_spacy = []
for i in range(0, len(data_to_process['noun_spacy'])-1):
    noun_list_spacy +=  data_to_process['noun_spacy'][i]

In [118]:
noun_list_spacy

['plan Scope management',
 'process',
 'Scope management plan document project product Scope',
 'key benefit process',
 'guidance direction Scope',
 'project',
 'process',
 'predefine point project',
 'Scope management plan',
 'component project program management plan',
 'Scope',
 'development Scope management plan',
 'project Scope',
 'analysis information',
 'project charter',
 'late approve subsidiary plan project management plan',
 'historical information',
 'organizational process',
 'relevant enterprise environmental factor',
 'enterprise environmental factor influence plan Scope management process',
 'organization culture',
 'infrastructure',
 'personnel administration',
 'marketplace condition',
 'organizational process asset influence plan Scope management process',
 'policy procedure',
 'historical information lesson',
 'repository',
 'data analysis technique',
 'use process',
 'alternative analysis',
 'requirement',
 'elaborate project product Scope',
 'product',
 'validate

In [119]:
from collections import Counter
counter = Counter(noun_list_spacy)
sorted_counter = counter.most_common()
sorted_counter

[('project', 38),
 ('lesson', 29),
 ('process', 27),
 ('figure', 23),
 ('work', 21),
 ('cost estimate', 21),
 ('speci', 20),
 ('resource', 18),
 ('use', 16),
 ('activity', 16),
 ('cost baseline', 16),
 ('contingency reserve', 16),
 ('project document', 15),
 ('work package', 14),
 ('key benefit process', 13),
 ('plan', 13),
 ('u', 13),
 ('requirement documentation', 13),
 ('example', 13),
 ('risk register', 13),
 ('schedule', 13),
 ('dependency', 13),
 ('alternative analysis', 12),
 ('requirement', 12),
 ('input process', 12),
 ('assumption log', 12),
 ('duration', 12),
 ('management reserve', 12),
 ('service', 11),
 ('requirement traceability matrix', 11),
 ('budget', 11),
 ('activity attribute', 11),
 ('project schedule', 11),
 ('activity list', 10),
 ('project team', 9),
 ('document', 9),
 ('duration estimate', 9),
 ('schedule model', 9),
 ('ac', 9),
 ('use process', 8),
 ('project Scope statement', 8),
 ('result', 8),
 ('decision', 8),
 ('constraint', 8),
 ('update result carry pro

In [120]:
len(verbs), len(verb_list_spacy)

(2817, 1811)

In [121]:
len(noun_list), len(noun_list_spacy)

(3668, 3585)

## 5.Extract Relation Between Entity 

### 1. SpaCy : Matchers

In [122]:
data_to_process['def'][0]

'plan Scope management is the process of creating a Scope management plan that documents how the project and product Scope will be defined, validated, and controlled. the key benefit of this process is that it provides guidance and direction on how Scope will be managed throughout the project. this process is performed once or at predefined points in the project. the Scope management plan is a component of the project or program management plan that describes how the Scope will be defined, developed, monitored, controlled, and validated. the development of the Scope management plan and the detailing of the project Scope begin with the analysis of information contained in the project charter  , the latest approved subsidiary plans of the project management plan  , historical information contained in the organizational process assets  and any other relevant enterprise environmental factors  '

In [123]:
data_to_process['sentence'][0][0]

'plan Scope management be process create Scope management plan document project product Scope will be define , validate , control .'

In [124]:
def relation_1(text):
    matcher_1 = Matcher(nlp.vocab)
    patterns_1 = [
        [{'POS': 'NOUN'}, {'POS':'VERB'} , {'POS':'NOUN'},{'POS':'NOUN'},{'POS':'NOUN'} ],
        [{'POS': 'NOUN'}, {'POS':'VERB'} , {'POS':'NOUN'},{'POS':'NOUN'}],
        [{'POS': 'NOUN'}, {'POS':'VERB'} , {'POS':'NOUN'}],
        
        [{'POS': 'NOUN'}, {'POS':'AUX'} , {'POS':'NOUN'},{'POS':'NOUN'},{'POS':'NOUN'} ],
        [{'POS': 'NOUN'}, {'POS':'AUX'} , {'POS':'NOUN'},{'POS':'NOUN'}],
        [{'POS': 'NOUN'}, {'POS':'AUX'} , {'POS':'NOUN'}]
    ]
    

    doc = nlp(text)
    matcher_1.add('rel_1',patterns_1)
    matches_1 = matcher_1(doc)

    matches_1 = [doc[start:end].text for match_id,start,end in matches_1]

    return matches_1

relation_1(data_to_process['sentence'][0][0])

['management be process']

In [125]:
def relation_2(text):
    matcher_2 = Matcher(nlp.vocab)
    patterns_2 = [
        [{'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS':'VERB'} , {'POS':'NOUN'},{'POS':'NOUN'},{'POS':'NOUN'} ],
        [{'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS':'VERB'} , {'POS':'NOUN'},{'POS':'NOUN'}],
        [{'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS':'VERB'} , {'POS':'NOUN'}],
        
        [{'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS':'AUX'} , {'POS':'NOUN'},{'POS':'NOUN'},{'POS':'NOUN'} ],
        [{'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS':'AUX'} , {'POS':'NOUN'},{'POS':'NOUN'}],
        [{'POS': 'NOUN'},{'POS': 'NOUN'}, {'POS':'AUX'} , {'POS':'NOUN'}]
    ]
    

    doc = nlp(text)
    matcher_2.add('rel_2',patterns_2)
    matches_2 = matcher_2(doc)

    matches_2 = [doc[start:end].text for match_id,start,end in matches_2]

    return matches_2

relation_2(data_to_process['sentence'][0][0])

[]

# Final DataFrame

# OWL File Creation

# Model Creation 